In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

health = pd.read_csv('Mental health Depression disorder Data.csv')

C:\Users\bekym\AppData\Local\Temp\ipykernel_24612\300513165.py:8: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  health = pd.read_csv('Mental health Depression disorder Data.csv')


In [2]:
health.head()

,index,Entity,Code,Year,Schizophrenia (%),Bipolar disorder (%),Eating disorders (%),Anxiety disorders (%),Drug use disorders (%),Depression (%),Alcohol use disorders (%)
0,0,Afghanistan,AFG,1990,0.16056,0.697779,0.101855,4.828830,1.677082,4.071831,0.672404
1,1,Afghanistan,AFG,1991,0.160312,0.697961,0.099313,4.829740,1.684746,4.079531,0.671768
2,2,Afghanistan,AFG,1992,0.160135,0.698107,0.096692,4.831108,1.694334,4.088358,0.670644
3,3,Afghanistan,AFG,1993,0.160037,0.698257,0.094336,4.830864,1.705320,4.096190,0.669738
4,4,Afghanistan,AFG,1994,0.160022,0.698469,0.092439,4.829423,1.716069,4.099582,0.669260


In [3]:
#print(health['Entity'].unique()) # choose 10 countries to analyze + US 

In [3]:
# builting the dataframe
    # create new df with countries we're interested in studying
    # fix the index 
    # added variable for whether it has universal healthcare or not
    
developed = health.loc[health['Entity'].isin(['United States','Japan','Argentina','Australia','United Kingdom', 'South Korea','Denmark','Canada','France','Taiwan','Uruguay','Germany'])]
developed.head()
developed = developed.reset_index(drop=True)

developed = developed.drop('index', axis=1)

# have universal care or practically universal care (for example may be small co-pays)
developed["Universal Care"] = np.nan

developed["Universal Care"] = np.where(developed['Entity'] == 'United States', 0,1) 



    


In [4]:
map_names = {'Entity': 'country',
             'Year' : 'year',
             'Schizophrenia (%)':'schizo', 
             'Bipolar disorder (%)':'bipolar',
             'Eating disorders (%)' : 'eating_dis',
             'Anxiety disorders (%)' : 'anxiety',
             'Drug use disorders (%)':'drug',
             'Depression (%)':'depres',
             'Alcohol use disorders (%)':'alcohol'}

developed.rename(columns=map_names,inplace=True)

In [5]:
developed

,country,Code,year,schizo,bipolar,eating_dis,anxiety,drug,depres,alcohol,Universal Care
0,Argentina,ARG,1990,0.194422,0.755674,0.323686,6.165921,0.919424,3.569180,1.918939,1
1,Argentina,ARG,1991,0.194184,0.756386,0.326904,6.174471,0.920323,3.579068,2.022040,1
2,Argentina,ARG,1992,0.194034,0.757139,0.330504,6.182707,0.921296,3.588855,2.116047,1
3,Argentina,ARG,1993,0.193973,0.757909,0.334286,6.190356,0.921620,3.597366,2.193307,1
4,Argentina,ARG,1994,0.194004,0.758686,0.338418,6.197271,0.921581,3.606913,2.246206,1
...,...,...,...,...,...,...,...,...,...,...,...
5947,Uruguay,URY,2013,121013.371501,NaN,NaN,NaN,NaN,NaN,NaN,1
5948,Uruguay,URY,2014,121976.987422,NaN,NaN,NaN,NaN,NaN,NaN,1
5949,Uruguay,URY,2015,122948.157402,NaN,NaN,NaN,NaN,NaN,NaN,1
5950,Uruguay,URY,2016,123877.363598,NaN,NaN,NaN,NaN,NaN,NaN,1
